In [1]:
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as mp
import matplotlib.animation as anim
import scipy.constants as sc
import numpy.random as rd
from jupyterthemes import jtplot

jtplot.reset()
mp.switch_backend("TkAgg")
mpl.interactive(True)
mp.rcParams["text.usetex"] = True
mp.rcParams["font.family"] = "serif"
mp.rcParams["figure.figsize"] = (7,7)

In [12]:
class Ising(object):
    
    def __init__(self, L=100, T=2.269, J=1, h=0):
        rd.seed(314)
        self.L = L
        self.s = rd.choice([1,-1], size=[L,L])
        self.T = T
        self.count = 0
        self.h = h
        self.J = J
        
    def get_mag(self):
        return np.sum(self.s)/self.L
        
    def metropolis(self, dE, y, x, s, T):
        if rd.random() < np.exp(dE/T):
            s[y,x] = -s[y,x]
        return s
    
    def deltaE(self,y,x):
        L,s = self.L, self.s
        per = np.empty([L+2,L+2], int)
        per[1:L+1,1:L+1] = s
        per[0,1:L+1] = s[L-1]
        per[L+1,1:L+1] = s[0]
        per[1:L+1,0] = s[:,L-1]
        per[1:L+1,L+1] = s[:,0]
        X = x+1
        Y = y+1
        s_j = per[Y-1,X] + per[Y+1,X] + per[Y,X-1] + per[Y,X+1]
        h_i = self.J*s_j + self.h
        dE = -2*s[y,x]*h_i
        return dE
        
    def on_key(self, event):
        key = event.key
        if key == "h":
            self.h += 1
        elif key == "g":
            self.h -= 1
        elif key == "t":
            self.T += 1
        elif key == "r":
            self.T /= 2
        elif key == "i":
            self.maxIters += 1
        elif key == "u":
            self.maxIters -= 1
        elif key == "j":
            self.J += 1
        elif key == "k":
            self.J -= 1
        elif key == "c":
            self.T = 2.269
            
    def continue_loop(self):
        while True:
            self.count += 1
            yield self.count
            
    def update_interactive(self, continue_loop):
        for _ in range(self.L):
            x,y = rd.randint(self.L), rd.randint(self.L)
            dE = self.deltaE(x,y)
            s = self.metropolis(dE,x,y,self.s,self.T)
            self.s = s
        self.im.set_data(self.s)
        self.data1_text.set_text(self.data1_template %(self.T, np.mean(self.h)))
        self.data2_text.set_text(self.data2_template %(self.L, self.get_mag(), self.J))
        return self.im, self.data1_text, self.data2_text
            
    def run_interactive(self):
        fig, (ax1,ax2) = mp.subplots(2,1, gridspec_kw={"height_ratios":[8,1]}, facecolor="k")
        self.data1_template = r"\textrm{\\ Ising ferromagnet: \\ temperature $T = %.2f$ \\ external field $h = %i$}"
        self.data2_template = r"\textrm{\\ lattice size $N = %i$ \\ magnetization $m = %.3f$ \\ coupling strength $J = %i$}"
        self.data1_text = ax2.text(0.15, 0.9, "", transform=ax2.transAxes, color="w", fontsize=12)
        self.data2_text = ax2.text(0.5, 0.9, "", transform=ax2.transAxes, color="w", fontsize=12)
        self.im = ax1.imshow(self.s, cmap="jet", animated=True)
        ax1.grid(False)
        ax1.axis("off")
        ax2.grid(False)
        ax2.axis("off")
        fig.canvas.mpl_connect("key_press_event", self.on_key)
        ani = anim.FuncAnimation(fig, self.update_interactive, self.continue_loop, interval=10, repeat=0)
        mp.tight_layout()
        mp.show(block=1)

In [14]:
sim = Ising(L=100, T=2.269, J=5, h=0)
sim.run_interactive()